# 初始化

In [12]:
import numpy as np
import pandas as pd

y = np.random.randint(2, size=(100, 1))
x = np.random.randint(10, size=(100, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

# Python版

In [2]:
def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result

In [33]:
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [4]:
%%timeit
target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 459 ms per loop


In [5]:
%%timeit
target_mean_v2(data, 'y', 'x')

100 loops, best of 3: 5.87 ms per loop


In [ ]:
result_1 = target_mean_v1(data, 'y', 'x')
result_2 = target_mean_v2(data, 'y', 'x')

diff = np.linalg.norm(result_1 - result_2)
print(diff)

0.0


In [3]:
%load_ext Cython

# Cython 直接执行纯python

In [10]:
%%cython -a
import numpy as np
import pandas as pd
cpdef target_mean_v3(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

  

In [13]:
%%timeit
target_mean_v3(data, 'y', 'x')

100 loops, best of 3: 5.45 ms per loop


# cython 版本

In [29]:
%%cython -a

import numpy as np
cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v4(data, y_name, x_name):
    cdef cnp.ndarray[long] X = data[x_name].values
    cdef cnp.ndarray[long] Y = data[y_name].values
    cdef Py_ssize_t N = X.shape[0] 
    v_dict = np.zeros(N, dtype=np.int)
    c_dict = np.zeros(N, dtype=np.int)
    cdef long[::1] value_dict = v_dict
    cdef long[::1] count_dict = c_dict

    cdef long[:] x_pt = X 
    cdef long[:] y_pt = Y
    cdef Py_ssize_t i
    cdef long x_id
    for i in range(N):
        x_id = x_pt[i]
        value_dict[x_id] += y_pt[i]
        count_dict[x_id] += 1
    
    result = np.zeros(N)
    cdef double[::1] result_pt = result
    for i in range(N): 
        x_id = x_pt[i]
        result_pt[i] = (value_dict[x_id] - y_pt[i]) / (count_dict[x_id] - 1)
    return result


In [35]:
%%timeit
target_mean_v4(data, 'y', 'x')

The slowest run took 19.93 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 12.5 µs per loop


In [34]:
result_2 = target_mean_v2(data, 'y', 'x')
result_4 = target_mean_v4(data, 'y', 'x')

diff = np.linalg.norm(result_4 - result_2)
print(diff)

0.0
